In [69]:
#!pip install scikit-learn
#!pip install pandas
#!pip install numpy
#!pip install shap

In [70]:
import shap

import pandas as pd  
import numpy as np  

# Pré-processamento e pipelines
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

# Modelos de ML
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR

# Validação e busca de hiperparâmetros
from sklearn.model_selection import train_test_split, KFold, GridSearchCV, cross_val_score

# Métricas
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error

# Regressão Random Forest
from sklearn.ensemble import RandomForestRegressor

## Carregando o DataSet e tratando as colunas para melhor uso nos modelos
- **Conversão da coluna `genres` em lista:** Separou os gêneros em uma lista para permitir a análise individual de cada gênero, facilitando a aplicação de técnicas como one-hot encoding.
- **Conversão da data de lançamento:** A coluna `release_date` foi convertida para o formato datetime, permitindo a extração de informações temporais relevantes como ano e mês de lançamento, que podem influenciar o desempenho do filme.
- **Explosão da coluna de gêneros:** Cada filme passou a ocupar uma linha para cada gênero associado, enriquecendo a granularidade dos dados e permitindo que modelos captem relações entre gêneros e receita.
- **Remoção de colunas desnecessárias:** Foram excluídas colunas como `id`, `title`, `status`, `release_date` e `genres` para evitar ruído e focar apenas nas variáveis relevantes para a modelagem.
- **Criação de novas features:** Foram criadas as colunas `release_year` e `release_month`, que podem capturar padrões sazonais ou tendências ao longo do tempo.

Essas transformações tornam o dataset mais adequado para algoritmos de ML, facilitando a codificação de variáveis categóricas, normalização de dados e extração de padrões relevantes para previsão de receita.

In [71]:
# Carregar o dataset
df = pd.read_csv('filmes_filtrados_sem_nulos.csv')

# Converter a coluna 'genres' para uma lista de gêneros
df['genres_names'] = df['genres'].apply(lambda x: x.split('-'))

# Tratando a data
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')
df['release_year'] = df['release_date'].dt.year
df['release_month'] = df['release_date'].dt.month

# Explodindo a coluna 'genres_names' para que cada gênero fique em uma linha separada
df = df.explode('genres_names').reset_index(drop=True)

# Retirando colunas desnecessárias para a análise
df = df.drop(columns=['id', 'title', 'status', 'release_date', 'genres'])
df.head()


,original_language,popularity,budget,revenue,runtime,vote_average,vote_count,genres_names,release_year,release_month
0,en,8763.998,129000000.0,352056482.0,116.0,7.079,1365.0,Action,2023,8
1,en,8763.998,129000000.0,352056482.0,116.0,7.079,1365.0,Science Fiction,2023,8
2,en,8763.998,129000000.0,352056482.0,116.0,7.079,1365.0,Horror,2023,8
3,en,5953.227,18000000.0,65675816.0,103.0,7.433,545.0,Horror,2023,4
4,en,5953.227,18000000.0,65675816.0,103.0,7.433,545.0,Mystery,2023,4


In [72]:
# Definindo as colunas de features e o target
features = ['original_language', 'popularity', 'budget', 'runtime', 'vote_average', 'vote_count', 'genres_names','release_year', 'release_month']
features_nominais = ['genres_names', 'original_language']
X = df[features]
y = df['revenue']
# Dividindo o dataset em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=87)

# Definindo o KFold para o cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=87)

In [73]:
# Pipeline para colunas nominais com StandardScaler
preprocessor_std = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), features_nominais),
        ('num', StandardScaler(), [col for col in features if col not in features_nominais])
    ]
)

pipeline_std = Pipeline([
    ('preprocess', preprocessor_std),
    ('regressor', KNeighborsRegressor())
])

# Parâmetros para GridSearchCV
param_grid = {
    'regressor__n_neighbors': [3, 5, 7, 9],
    'regressor__weights': ['uniform', 'distance']
}

grid = GridSearchCV(
    pipeline_std,
    param_grid,
    cv=kf,
    scoring='neg_mean_squared_error',
    n_jobs=-1
)

grid.fit(X_train, y_train)

print("Melhores parâmetros:", grid.best_params_)
print("Melhor score (neg MSE):", grid.best_score_)


Melhores parâmetros: {'regressor__n_neighbors': 5, 'regressor__weights': 'distance'}
Melhor score (neg MSE): -7029817677680000.0


In [74]:
# Pipeline para colunas nominais com StandardScaler e SVM + GridSearchCV para o parâmetro C
preprocessor_standard = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), features_nominais),
        ('num', StandardScaler(), [col for col in features if col not in features_nominais])
    ]
)

pipeline_svm = Pipeline([
    ('preprocess', preprocessor_standard),
    ('regressor', SVR())
])

# Parâmetros para GridSearchCV (analisando o parâmetro C)
param_grid_svm = {
    'regressor__C': [0.1, 1, 10, 100]
}

grid_svm = GridSearchCV(
    pipeline_svm,
    param_grid_svm,
    cv=kf,
    scoring='neg_mean_squared_error',
    n_jobs=-1
)

grid_svm.fit(X_train, y_train)

print("Melhor parâmetro C:", grid_svm.best_params_)
print("Melhor score (neg MSE):", grid_svm.best_score_)

Melhor parâmetro C: {'regressor__C': 100}
Melhor score (neg MSE): -3.704826748368972e+16


# Usando de Log nas Features "budget" e "revenue": 
O uso da transformação logarítmica nas variáveis "budget" e "revenue" é uma prática comum em problemas de regressão envolvendo valores financeiros, pois esses dados costumam apresentar grande dispersão e distribuição assimétrica (com muitos valores extremos). Ao aplicar o log, reduzimos o impacto de outliers, aproximamos a distribuição dos dados de uma normal e facilitamos o aprendizado dos modelos de machine learning. Isso pode resultar em previsões mais estáveis e métricas de avaliação mais realistas, além de melhorar a capacidade do modelo de capturar relações proporcionais entre as variáveis.

In [75]:
# Criar uma cópia do dataframe
df_log = df.copy()

# Aplicar log1p para lidar com valores zero
df_log['budget'] = np.log1p(df_log['budget'])

# X e y
X_log = df_log[features]
y = df['revenue'] 

# Dividir em treino e teste
X_train_log, X_test_log, y_train, y_test = train_test_split(X_log, y, test_size=0.3, random_state=87)

# Log1p apenas no y_train
y_train_log = np.log1p(y_train)

In [76]:
# Pipeline para colunas nominais com StandardScaler e SVM + GridSearchCV para o parâmetro C, usando dados com log
pipeline_svm_log = Pipeline([
    ('preprocess', preprocessor_standard),
    ('regressor', SVR())
])

param_grid_svm_log = {
    'regressor__C': [0.1, 1, 10, 100]
}

grid_svm_log = GridSearchCV(
    pipeline_svm_log,
    param_grid_svm_log,
    cv=kf,
    scoring='neg_mean_squared_error',
    n_jobs=-1
)

grid_svm_log.fit(X_train_log, y_train_log)

print("Melhor parâmetro C (log):", grid_svm_log.best_params_)
print("Melhor score (neg MSE, log):", grid_svm_log.best_score_)


Melhor parâmetro C (log): {'regressor__C': 10}
Melhor score (neg MSE, log): -1.902796595714765


## Regressão com Random Forest com budget, popularity, runtime, vote_average e vote_count

O modelo de Regressão com Random Forest é um tipo de ensemble learning que combina várias árvores de decisão para melhorar a precisão e reduzir o risco de overfitting. Ele é especialmente útil quando há relações não lineares entre as variáveis e quando temos outliers, como é o caso com valores de budget e revenue. Esse modelo é robusto, simples de ajustar e interpretável em termos de importância de cada variável.

In [77]:
features = [
    'original_language', 'popularity', 'budget', 'runtime', 
    'vote_average', 'vote_count', 'genres_names', 
    'release_year', 'release_month'
]
features_nominais = ['genres_names', 'original_language']

# com log
X = df[features]
y_log = np.log1p(df['revenue'])

X_train, X_test, y_train_log, y_test_log = train_test_split(X, y_log, test_size=0.3, random_state=87)
preprocessor_rf = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), features_nominais),
        ('num', StandardScaler(), [col for col in features if col not in features_nominais])
    ]
)
pipeline_rf_log = Pipeline([
    ('preprocess', preprocessor_rf),
    ('regressor', RandomForestRegressor(n_estimators=100, random_state=87))
])
pipeline_rf_log.fit(X_train, y_train_log)
y_pred_log = pipeline_rf_log.predict(X_test)
y_pred_log_original = np.expm1(y_pred_log)
y_test_log_original = np.expm1(y_test_log)
mse_log = mean_squared_error(y_test_log_original, y_pred_log_original)
r2_log = r2_score(y_test_log_original, y_pred_log_original)

print("Random Forest (com log)")
print("MSE:", mse_log)
print("R²:", r2_log)

# sem log
y = df['revenue']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=87)

pipeline_rf = Pipeline([
    ('preprocess', preprocessor_rf),
    ('regressor', RandomForestRegressor(n_estimators=100, random_state=42))
])

pipeline_rf.fit(X_train, y_train)
y_pred = pipeline_rf.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Random Forest (sem log)")
print("MSE:", mse)
print("R²:", r2)


Random Forest (com log)
MSE: 1958740565932538.5
R²: 0.9438750912767355
Random Forest (sem log)
MSE: 1305474796009980.0
R²: 0.9625934873454263


In [78]:
# Pipeline para colunas nominais com StandardScaler e KNN, usando dados com log
pipeline_knn_log = Pipeline([
    ('preprocess', preprocessor_standard),
    ('regressor', KNeighborsRegressor())
])

param_grid_knn_log = {
    'regressor__n_neighbors': [3, 5, 7, 9],
    'regressor__weights': ['uniform', 'distance']
}

grid_knn_log = GridSearchCV(
    pipeline_knn_log,
    param_grid_knn_log,
    cv=kf,
    scoring='neg_mean_squared_error',
    n_jobs=-1
)

grid_knn_log.fit(X_train_log, y_train_log)

print("Melhores parâmetros (KNN log):", grid_knn_log.best_params_)
print("Melhor score (neg MSE, KNN log):", grid_knn_log.best_score_)

Melhores parâmetros (KNN log): {'regressor__n_neighbors': 9, 'regressor__weights': 'distance'}
Melhor score (neg MSE, KNN log): -2.3056250963677756


In [80]:
modelos = {
    'SVM (StandardScaler)': grid_svm,
    'SVM (StandardScaler com log)': grid_svm_log,
    'KNN (StandardScaler)': grid,
    'KNN (StandardScaler com log)': grid_knn_log,
    'Random Forest (StandardScaler)': pipeline_rf,
    'Random Forest (StandardScaler com log)': pipeline_rf_log,
}

resultados = []

for nome, modelo in modelos.items():
    if 'com log' in nome:
        y_pred_log = modelo.predict(X_test_log)
        y_pred = np.expm1(y_pred_log)
        mse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
    else:
        y_pred = modelo.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
    resultados.append({'Modelo': nome, 'MSE': mse, 'R²': r2})

resultados_df = pd.DataFrame(resultados)
print(resultados_df)

                                   Modelo           MSE        R²
0                    SVM (StandardScaler)  3.941641e+16 -0.129421
1            SVM (StandardScaler com log)  7.976347e+15  0.771449
2                    KNN (StandardScaler)  7.062214e+15  0.797642
3            KNN (StandardScaler com log)  1.041102e+16  0.701687
4          Random Forest (StandardScaler)  1.305475e+15  0.962593
5  Random Forest (StandardScaler com log)  3.965721e+16 -0.136320
